<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<h1><font size=4>Trabajo Fin de Master (TFM)</font></h1>
<br>
<h2><font size=6>WiDS Datathon 2024 - Challenge 2</font></h2>
<h3><font size=5>Modelos de regresión para estimación del periodo de diagnóstico metastático</font></h3>
<h3><font size=5>Parte 2 - Modelos de Regresión</font></h3>
<br>
<h1><font size=4>Alumna: Luna Jiménez Fernández</font></h1>
<br>



<div align="right">
<font size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font size=3>Universidad de Castilla-La Mancha</font>
</div>

<br>

---

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Array manipulation libraries
import numpy as np
import pandas as pd

# Regression models
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline


# Importing visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [4]:
# Seed for random experiments - 7 is the number
RANDOM_SEED = 777

En la primera libreta se realizó un **análisis exploratorio de datos** exhaustivo para entender en profundidad el comportamiento del conjunto de datos de interés - el **segundo desafío** del **Women in Data Science (*WiDS*) Datathon** del año 2024, disponible en el [siguiente enlace](https://www.kaggle.com/competitions/widsdatathon2024-challenge2/overview).

Tras este estudio, el objetivo de la siguiente libreta es tanto la **construcción de modelos de regresión** capaces de predecir el **tiempo de diagnóstico de la metástasis** a partir de los atributos seleccionados, como la **evaluación** de estos con el fin de estudiar si resultan de utilidad y si - como se planteó - los **atributos geográficos, socioeconómicos y climáticos** juegan algún papel relevante en la estimación de los valores.

---

# Índice

* [3. Selección de atributos y pre-procesamiento](#section3)
    * [3.1. Carga y particionamiento del conjunto de datos](#section3-1)
    * [3.2. Selección de atributos](#section3-2)
    * [3.3. Pre-procesamiento de los atributos seleccionados](#section3-3)
* [4. Selección de modelos de regresión e hiperparámetros](#section4)
* [5. Experimentación](#section5)
* [6. Análisis de resultados](#section6)
* [7. Conclusiones](#section7)
---

<a id="section3"></a>

# 3. Selección de atributos y pre-procesamiento

Tras finalizar el **análisis exploratorio de datos** en la libreta anterior, el siguiente paso en el proceso de ciencia de datos es el **preprocesamiento de la información** - para ser utilizada posteriormente por modelos de regresión, con el fin de predecir el tiempo de diagnóstico de la metástasis.

Concretamente, en este apartado se realizan las siguientes preparaciones:
- **Cargar y particionar** los conjuntos de datos en **entrenamiento**, **validación** y **test**.
- **Seleccionar el subconjunto de atributos** que van a ser utilizados durante la experimentación.
- **Preparar las *pipelines*** encargadas de transformar los datos crudos en datos listos para ser utilizados por los modelos posteriores.

---

<a id="section3-1"></a>

## 3.1. Carga y particionamiento del conjunto de datos

Durante el análisis exploratorio de datos se trabajó únicamente sobre el **conjunto de entrenamiento** - con el fin de evitar cualquier posible fuga de datos al estudiar el conjunto de test. Ahora bien, el desafio en Kaggle ofrece **dos conjuntos de datos**:
- `train.csv`: El **conjunto de entrenamiento**, con **150 atributos** y los valores de la **variable objetivo** (el tiempo de diagnóstico) asociados a cada instancia.
- `test.csv`: El **conjunto de test**, conteniendo únicamente los **150 atributos** sin los valores de la variable objetivo.

El primer paso, por tanto, consiste en **cargar ambos conjuntos de datos** como *DataFrames*:

In [11]:
# Loading the CSV files
df_train = pd.read_csv("data/train.csv", index_col="patient_id")
df_test = pd.read_csv("data/test.csv", index_col="patient_id")

# Display a small sample of both datasets to show that they have been properly loaded
print(f"Training size: {df_train.shape}")
display(df_train.sample(5))
print(f"Test size: {df_test.shape}")
display(df_test.sample(5))

Training size: (13173, 151)


,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period
patient_id,,,,,,,,,,,,,,,,,,,,,
447313,NaN,COMMERCIAL,CA,945,West,Pacific,58,F,24.98,1744,...,56.99,59.82,67.89,73.17,69.72,67.93,64.00,56.76,49.11,168
247903,Other,COMMERCIAL,CA,902,West,Pacific,60,F,27.46,C50112,...,63.40,63.22,67.67,76.15,75.44,71.31,69.03,65.45,59.37,62
451011,NaN,MEDICARE ADVANTAGE,NC,282,South,South Atlantic,91,F,NaN,C50812,...,58.05,73.86,79.46,80.29,79.48,79.25,64.62,48.12,45.52,12
771602,NaN,MEDICARE ADVANTAGE,OH,454,Midwest,East North Central,82,F,NaN,1749,...,45.96,69.80,73.14,74.20,74.31,70.71,55.25,37.11,34.94,270
859408,White,NaN,WA,989,West,Pacific,49,F,NaN,C50311,...,47.67,62.79,62.56,71.22,69.50,58.58,47.77,38.16,34.52,49


Test size: (5646, 150)


,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
patient_id,,,,,,,,,,,,,,,,,,,,,
985624,Black,NaN,CO,800,West,Mountain,49,F,NaN,C50911,...,43.24,48.85,61.47,72.59,74.94,72.44,67.41,49.12,38.81,33.29
439802,White,MEDICAID,CA,926,West,Pacific,53,F,29.38,C50412,...,60.18,63.97,64.32,67.87,77.78,77.55,73.11,69.83,64.83,58.61
816382,NaN,COMMERCIAL,WA,993,West,Pacific,57,F,NaN,C50111,...,43.72,51.48,64.84,65.76,74.90,72.62,61.87,50.80,40.06,37.40
590527,NaN,COMMERCIAL,OK,740,South,West South Central,63,F,NaN,C50412,...,50.28,52.22,74.41,78.66,81.48,78.65,73.28,59.24,43.23,39.63
878530,Asian,MEDICAID,NY,113,Northeast,Middle Atlantic,60,F,NaN,C50912,...,37.95,47.68,65.17,70.33,77.22,77.83,69.93,55.92,42.16,37.26


Ahora bien, debido al proceso que se va a seguir durante el entrenamiento de los modelos (**selección de hiperparámetros**, **selección de modelos** y **evaluación**), utilizar directamente los conjuntos de datos descritos podría llevar a un problema de **fuga de datos** - al usar el mismo conjunto de datos para entrenar los modelos y evaluar sus hiperparámetros.

Para evitar esto, se va a dividir el conjunto de entrenamiento en dos - un conjunto de **entrenamiento** y uno de **validación**  -, siendo la distribución final la siguiente:
- **Entrenamiento**: El conjunto de entrenamiento cumple dos tareas - tanto el **entrenamiento de los modelos de regresión** propuestos como el **ajuste de hiperparámetros de los mismos** a través de una validación cruzada.
- **Validación**: Una vez se tienen los modelos entrenados, el conjunto de validación será utilizado para **seleccionar el mejor modelo de forma honesta** - utilizando un conjunto de datos que no han utilizado durante el entrenamiento para evitar sesgos o fugas de datos.
- **Test**: Finalmente, se utilizará el conjunto de test para **evaluar el rendimiento real** del modelo seleccionado a través de la validación - utilizando una plataforma externa (***Kaggle***) para medir este rendimiento. 

Además, todos estos conjuntos de datos se van a fraccionar en **atributos** (`X`) y **variable objetivo** (`y`) para seguir el estándar de `scikit-learn`.

In [13]:
# Split the datasets into training, validation and test
# Train / Val
X, y = df_train.drop(columns="metastatic_diagnosis_period"), df_train["metastatic_diagnosis_period"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

# Test
X_test = df_test

# Display the information about each dataset - to ensure that it has been loaded and partitioned correctly
print("ENTRENAMIENTO:")
print(f"\t-Atributos: {X_train.shape}")
print(f"\t-Variable objetivo: {y_train.shape}")
print("VALIDACIÓN:")
print(f"\t-Atributos: {X_val.shape}")
print(f"\t-Variable objetivo: {y_val.shape}")
print("TEST:")
print(f"\t-Atributos: {X_test.shape}")

ENTRENAMIENTO:
	-Atributos: (9879, 150)
	-Variable objetivo: (9879,)
VALIDACIÓN:
	-Atributos: (3294, 150)
	-Variable objetivo: (3294,)
TEST:
	-Atributos: (5646, 150)


---

<a id="section3-2"></a>

## 3.2. Selección de atributos

Como se observó durante el análisis exploratorio, no tendría sentido utilizar directamente el **conjunto de datos completos** para el entrenamiento de modelos:
- La **dimensionalidad del conjunto de datos** - con 150 atributos en total - es excesiva para la cantidad de datos disponible, lo que podría llevar a sobreajustes.
- Algunos atributos tienen **una cantidad excesiva de posibles valores** - que se puede traducir, de nuevo, en sobreajustes del modelo al no tener suficientes datos para aprender adecuadamente las relaciones.
- La **amplia mayoría de atributos son irrelevantes** para la variable objetivo - ya sea por su baja calidad o por la poca correlación que tienen con la variable objetivo.

Por tanto, es necesario realizar una **selección de un subconjunto de atributos** para reducir la dimensionalidad y cribar los atributos que no sean relevantes para la predicción. Para buscar este subconjunto, se proponen varias opciones:

### 3.2.1. Selección manual

Durante el análisis exploratorio de datos se ha realizado un análisis exhaustivo de los datos - tanto su **comportamiento** como su **relevancia** y las **transformaciones** que serían necesarias para utilizarse.

A partir de las conclusiones extraidas, se obtiene el siguiente **conjunto de atributos** - representando los atributos más relevantes estudiados dentro del conjunto de datos, junto a las **transformaciones a aplicar** sobre estos datos:
- **Código de diagnóstico del cancer de mama (`breast_cancer_diagnosis_code`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Código de diagnóstico del cancer metastático (`metastatic_cancer_diagnosis_code`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Estado de residencia del paciente (`patient_state`):** Variable categórica.
    - Debido al gran número de posibles valores, es necesario **agrupar los valores menos frecuentes**.
- **Raza del paciente (`patient_race`):** Variable categórica.
    - Se **agrupan los valores perdidos** bajo un único valor - `Unknown`.
- **Tipo de seguro médico del paciente (`payer_type`):** Variable categórica.
    - Se **agrupan los valores perdidos** bajo un único valor - `UNKNOWN`.

In [14]:
attributes_manual = [
    "breast_cancer_diagnosis_code",
    "metastatic_cancer_diagnosis_code",
    "patient_race",
    "payer_type",
    "patient_state"
]

Estas transformaciones se han elegido en base a los **test estadísticos** que se realizaron durante el análisis exploratorio:
- La **agrupación de los valores** en las variables de alta dimensionalidad aumenta la significación estadística, al reducirse el número de valores con un número demasiado bajo de instancias.
- La **sustitución de valores perdidos** mejora el rendimiento en los atributos donde el número de valores perdidos es excesivo.

A su vez, se ha optado por descartar los siguientes atributos:
- **Edad (`patient_age`) y IMC (`bmi`) del paciente**: Variables numéricas sin correlación con la variable objetivo.
- **Región (`Region`) y división (`Division`) del paciente**: Variables categóricas con poca relevancia, y ya representadas por otra variable más significativa (`patient_state`).
- **Código zip del paciente (`patient_zip3`)**: Variable categórica ya representada por otra variable (`patient_state`) con dimensionalidad excesiva.
- **Todas las variables geográficas, socioeconómicas y climáticas**: 136 atributos numéricos sin correlación con la variable objetivo.

---

<a id="section3-3"></a>

## 3.3. Pre-procesamiento de los atributos seleccionados

<a id="section4"></a>

# 4. Selección de modelos de regresión e hiperparámetros

<a id="section5"></a>

# 5. Experimentación

<a id="section6"></a>

# 6. Análisis de resultados